In [ ]:
#Mon ini (SON)
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/M1_S2/RITAL/projet2
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

!pip install python-terrier
import pandas as pd

import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

In [3]:
import string
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer


def prepretraiter(query, ponctuation=True, stem=True, lemme=False, stopword=True):
    """
    Préprétraite une query
    """
    #wn_lemmatizer = nltk.stem.WordNetLemmatizer()
    stemmer = PorterStemmer()
    wordnet_lemmatizer = WordNetLemmatizer()
    stopwords_en = stopwords.words('english')
    punc = string.punctuation+'\n\r\t'
    q_traite = []
    for q in query:
        q = q[0].lower()
        if ponctuation:
          q = q.translate(str.maketrans(punc, ' '*len(punc)))
        if stopword:
          q = word_tokenize(q)
          q = ' '.join([w for w in q if w not in stopwords_en])
        #q = ' '.join(map(lambda x: wn_lemmatizer.lemmatize(x), word_tokenize(q)))
        if lemme:
          q = ' '.join(map(lambda x: wordnet_lemmatizer.lemmatize(x), word_tokenize(q)))
        if stem:
          q = ' '.join(map(lambda x: stemmer.stem(x), word_tokenize(q)))
        q_traite.append(q)
    return q_traite

def applique_pretraite_passage (df, ponctuation=True, stem=True,lemme=False, stopword=True) :
    """
    Applique le préprétraitement de query à un dataframe
    """
    liste_query=df[['text']].values.tolist()
    liste_query=prepretraiter(liste_query, ponctuation=ponctuation, stem=stem, stopword=stopword)
    df['text']=liste_query
    return df

def applique_pretraite_query (df, ponctuation=True, stem=True,lemme=False, stopword=True) :
    """
    Applique le préprétraitement de query à un dataframe
    """
    liste_query=df[['query']].values.tolist()
    liste_query=prepretraiter(liste_query, ponctuation=ponctuation, stem=stem, stopword=stopword)
    df['query']=liste_query
    df = df.astype({"qid": str, "query": object})
    
    return df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Préparation de données

In [2]:
# chargement du json en dataframe
with open('WikiPassageQA/document_passages.json', 'r') as f:
        data_row = json.load(f)
data={"DocumentID" : [int(k0) for k0,v0 in data_row.items() for k1,v1 in v0.items()],
    "docno" : [k0+','+k1 for k0,v0 in data_row.items() for k1,v1 in v0.items()],
      "text"  : [v1 for k0,v0 in data_row.items() for k1,v1 in v0.items()]}
docs_df = pd.DataFrame(data)

In [4]:
docs_df = applique_pretraite_passage (docs_df, ponctuation=True, stem=False,lemme=True, stopword=True)
docs_df

,DocumentID,docno,text
0,344,"344,1",townspeople lived chartered towns burghers opp...
1,344,"344,0",municipal corporation legal term local governi...
2,344,"344,3",standard varies state state according laws pas...
3,344,"344,2",municipal corporations must also abide organic...
4,344,"344,5",panchayati raj system interacts directly state...
...,...,...,...
50607,479,"479,18",dialects also divided two main chronological c...
50608,479,"479,31",official language moscow novgorod later growin...
50609,479,"479,30",soon followed adoption christianity 988 introd...
50610,479,"479,33",knam va kosk z stu tcit political upheavals ea...


## Query

In [6]:
df_train = pd.read_csv("WikiPassageQA/train.tsv",sep='\t')
df_test = pd.read_csv("WikiPassageQA/test.tsv",sep='\t')
df_dev = pd.read_csv("WikiPassageQA/dev.tsv",sep='\t')
#Question None
df_train = df_train[df_train['Question'] != '{}']
df_test = df_test[df_test['Question'] != '{}']
df_dev = df_dev[df_dev['Question'] != '{}']

queries_train=df_train[["QID", "Question"]]
queries_train.columns = ['qid', 'query']
queries_test=df_test[["QID", "Question"]]
queries_test.columns = ['qid', 'query']
queries_dev=df_dev[["QID", "Question"]]
queries_dev.columns = ['qid', 'query']
df_query = pd.concat([df_train, df_test, df_dev])
df_complete = df_query.merge(docs_df, on =["DocumentID"])
df_complete

,QID,Question,DocumentID,DocumentName,RelevantPassages,docno,text
0,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,42",claimed major credit election fernando collor ...
1,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,48",evangelicalism major force anglican church 180...
2,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,43",rich poor remained traditional catholics evang...
3,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,49",evangelical alliance formed 1846 first ecumeni...
4,3086,What is the role of conversionism in Evangelic...,672,Evangelicalism.html,4,"672,24",1737 whitefield become national celebrity engl...
...,...,...,...,...,...,...,...
244014,2570,How was the kalmar Union formed?,553,Norway.html,"18,19,20,21","553,72",budgetary rule spend 4 fund year august 2014 g...
244015,2570,How was the kalmar Union formed?,553,Norway.html,"18,19,20,21","553,71",norway fifth largest oil exporter third larges...
244016,2570,How was the kalmar Union formed?,553,Norway.html,"18,19,20,21","553,70",2011 28 state revenues generated petroleum ind...
244017,2570,How was the kalmar Union formed?,553,Norway.html,"18,19,20,21","553,79",two prominent e6 going north south entire coun...


# Jugement de pertinence

In [7]:
relevant = []
for row in df_complete.iterrows():
  passage = row[1]['RelevantPassages'].split(",")
  doc_passage = row[1]['docno'].split(",")[1] # doc,pass -> [1]
  if doc_passage in passage:
    relevant.append(1)
  else:
    relevant.append(0)

df_complete["label"] = relevant
qrel = df_complete[["QID", "docno","label"]]
qrel.columns = ['qid', 'docno', 'label']
qrel = qrel.astype({"qid": str, "docno": object, "label":object}) #qid en string pour pt.Experiment
qrel

,qid,docno,label
0,3086,"672,42",0
1,3086,"672,48",0
2,3086,"672,43",0
3,3086,"672,49",0
4,3086,"672,24",0
...,...,...,...
244014,2570,"553,72",0
244015,2570,"553,71",0
244016,2570,"553,70",0
244017,2570,"553,79",0


In [8]:
stemme = False
lemme=True
stpword = True
queries_train=applique_pretraite_query(queries_train,ponctuation=True, stem=stemme,lemme=lemme, stopword=stpword)
queries_test=applique_pretraite_query(queries_test,ponctuation=True, stem=stemme,lemme=lemme, stopword=stpword)
queries_dev=applique_pretraite_query(queries_dev,ponctuation=True, stem=stemme,lemme=lemme, stopword=stpword)

queries_test

<ipython-input-3-2432fa1339f8>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['query']=liste_query
<ipython-input-3-2432fa1339f8>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['query']=liste_query
<ipython-input-3-2432fa1339f8>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

,qid,query
0,449,iraq role political unstabilization
1,1140,warmest years global warming
2,782,olympic games broadcasted
3,1421,election turnout 2008
4,3738,secularism misinterpreted america
...,...,...
411,2522,church mean fallen nature
412,2851,history eurasia
413,3705,nicknames missouri
414,799,endoglossic exoglossic languages coexist nigeria


# BERT

In [ ]:
# Paquets pour BERT
!pip install faiss-gpu
!apt install --upgrade libomp-dev

!pip install --upgrade transformers==3.0.2

In [ ]:
# plugin

!pip install --upgrade git+https://github.com/terrierteam/pyterrier_colbert.git

In [12]:
!rm -rf /content/colbert_index

import pyterrier_colbert.indexing
# BERT checkpoint
checkpoint="http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip"
colbert_indexer = pyterrier_colbert.indexing.ColBERTIndexer(checkpoint=checkpoint, 
                                                            index_root="/content",
                                                            index_name="colbert_index",
                                                            chunksize=3)

In [13]:
from typing import Union, Tuple, Iterator, Dict, Any, List

class GeneratorLen(object):
    def __init__(self, gen, length):
        self.gen = gen
        self.length = length

    def __len__(self): 
        return self.length

    def __iter__(self):
        return self.gen

def custom_get_corpus_iter(df: pd.DataFrame) -> Iterator[Dict[str,Any]]:
  len_total = df.shape[0]
  def gen():
    for _, row in df.iterrows():
      yield {"docno": row["docno"], "text": row["text"]}

  return GeneratorLen(gen(), len_total)

In [14]:
colbert_indexer.index(custom_get_corpus_iter(docs_df))

[May 18, 17:21:54] [0] 		 #> Local args.bsize = 128
[May 18, 17:21:54] [0] 		 #> args.index_root = /content
[May 18, 17:21:54] [0] 		 #> self.possible_subset_sizes = [69905]


Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ColBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ColBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.weight']
You should probably TRAI

[May 18, 17:21:57] #> Loading model checkpoint.
[May 18, 17:21:57] #> Loading checkpoint http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip


Downloading: "http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/colbert_model_checkpoint.zip" to /root/.cache/torch/hub/checkpoints/colbert_model_checkpoint.zip
100%|██████████| 1.11G/1.11G [00:11<00:00, 105MB/s] 
/usr/local/lib/python3.10/dist-packages/torch/hub.py:665: UserWarning: Falling back to the old format < 1.6. This support will be deprecated in favor of default zipfile format introduced in 1.6. Please redo torch.save() to save it in the new zipfile format.
  warnings.warn('Falling back to the old format < 1.6. This support will be '


[May 18, 17:22:18] #> checkpoint['epoch'] = 0
[May 18, 17:22:18] #> checkpoint['batch'] = 44500






[May 18, 17:22:19] #> Note: Output directory /content already exists




[May 18, 17:22:19] #> Creating directory /content/colbert_index 


[May 18, 17:24:21] [0] 		 #> Completed batch #0 (starting at passage #0) 		Passages/min: 24.9k (overall),  25.4k (this encoding),  99507.2M (this saving)
[May 18, 17:24:22] [0] 		 [NOTE] Done with local share.
[May 18, 17:24:22] [0] 		 #> Joining saver thread.
[May 18, 17:24:23] [0] 		 #> Saved batch #0 to /content/colbert_index/0.pt 		 Saving Throughput = 1.9M passages per minute.

#> num_embeddings = 4861412
[May 18, 17:24:23] #> Starting..
[May 18, 17:24:23] #> Processing slice #1 of 1 (range 0..1).
[May 18, 17:24:23] #> Will write to /content/colbert_index/ivfpq.100.faiss.
[May 18, 17:24:23] #> Loading /content/colbert_index/0.sample ...
#> Sample has shape (243070, 128)
[May 18, 17:24:23] Preparing resources for 1 GPUs.
[May 18, 17:24:23] #> Training with the vectors...
[May 18, 17:24:23] #> Training now (using 1 GPUs)...
879.6283919811249
1

In [15]:
colbert_retriever = colbert_indexer.ranking_factory().end_to_end()

pt.Experiment(
    [colbert_retriever], 
    queries_test,
    qrel,
    eval_metrics=["map", "recip_rank", "P_5","P_10","ndcg", "recall_5", "recall_10", "recall_20" ],
    names=['ColBERT'],
)

[May 18, 17:40:01] #> Loading the FAISS index from /content/colbert_index/ivfpq.100.faiss ..
[May 18, 17:40:01] #> Building the emb2pid mapping..
[May 18, 17:40:01] len(self.emb2pid) = 4861412
Loading reranking index, memtype=mem


Loading index shards to memory:   0%|          | 0/1 [00:00<?, ?shard/s]

,name,map,recip_rank,P_5,P_10,ndcg,recall_5,recall_10,recall_20
0,ColBERT,0.467077,0.555638,0.163942,0.09976,0.599076,0.552244,0.645873,0.711859
